In [9]:
from torchvision import models, transforms
from PIL import Image
import torch
import os

In [10]:
# Load pretrained ResNet18
model = models.resnet18(weights='IMAGENET1K_V1')
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [11]:
# Define transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

In [12]:
# Folders
classes = ['Healthy', 'Diseased']
dataDirectory = 'data'

correct = 0
total = 0

In [13]:

for label in classes:
    folder = os.path.join(dataDirectory, label)
    
    for img_name in os.listdir(folder):
        
        # Open the image
        img_path = os.path.join(folder, img_name)
        img = Image.open(img_path).convert('RGB')
        
        # Feed it through the model
        x = transform(img).unsqueeze(0)
        with torch.no_grad():
            out = model(x)
        
        # Obtain the prediction
        pred = out.argmax(dim=1).item()
        
        # Determine its class
        if classes[pred % 2] == label:  # just for rough demo
            correct += 1
        total += 1

print("Accuracy:", correct / total)

Accuracy: 0.4993946731234867
